In [ ]:
# Initalize important stuff
import xspec as xs
import matplotlib.pyplot as plt, numpy as np, pandas as pd
import matplotlib.ticker as mticker
import seaborn as sns
sns.set(style='dark')
import time
from IPython.display import clear_output
def clear(): time.sleep(0.1), clear_output()
    
xs.AllModels.lmod("acx", "/home/vatsal/soft/atomdbcx/xspec")

In [ ]:
# Load XMM data
xs.AllData.clear()
d = xs.Spectrum('eq080420_src_bin5.ds')
xs.Plot.xAxis = "keV"
d.ignore("**-**")
d.notice("0.2-1")

clear()

In [ ]:
# Plot the XMM data
xs.Plot('data')
xmm_df = pd.DataFrame({'x_xmm':xs.Plot.x(), 'y_xmm':xs.Plot.y(), 'xerr_xmm':xs.Plot.xErr(), 'yerr_xmm':xs.Plot.yErr()})

plt.figure(figsize=(10,4))
plt.errorbar(x=xmm_df.x_xmm, y=xmm_df.y_xmm, xerr=xmm_df.xerr_xmm, yerr=xmm_df.yerr_xmm, fmt='.', elinewidth=0.6)
plt.title("XMM equator solar min 2020 (bin5)")
plt.xlabel('Energy / keV')
plt.ylabel('Counts / keV / s')
plt.grid(color='w')

# plt.savefig('xmm-solar-min-equator', bbox_inches='tight')

In [ ]:
# Load models
m = xs.Model('apec')

m(1).values = 0.12  		    # plasma temp kT
m(2).values = 1 			# Abundance
m(3).values = 0 			# Redshift
m(4).values = 5.1e-06			# Norm

m(1).frozen = True
m(2).frozen = True
m(4).frozen = True

# xs.Fit.nIterations = 50
# xs.Fit.perform()

xs.Plot('data')
folded_model = xs.Plot.model()

In [ ]:
# Saving and restoring our model

!rm solar_min.xcm                    # Remove previous saved model
xs.Xset.save('solar_min.xcm')        # Save current model
xs.AllModels.clear()
xs.Xset.restore('solar_min.xcm')     # Restore current model
clear()

In [ ]:
# Make fake spectrum in LEM
response_file = 'lem_09ev_110422.rmf'
arf_file      = 'lem_110422.arf'
exposure_time = '144000'

!rm solar_min.fak                # Remove previous saved fake spectrum
xs.AllData.clear()
fs = xs.FakeitSettings(response=response_file, arf=arf_file, exposure=exposure_time)
fs.fileName = 'solar_min.fak'    # Save current saved fake spectrum
xs.AllData.fakeit(1,fs)

# xs.Fit.nIterations = 50
# xs.Fit.perform()
clear()

In [ ]:
# Plot LEM spectra
xs.Plot('data')
lem_df = pd.DataFrame({'x_lem':xs.Plot.x(), 'y_lem':xs.Plot.y(), 'xerr_lem':xs.Plot.xErr(), 'yerr_lem':xs.Plot.yErr()})

fig = plt.figure()

ax1 = fig.add_axes([0, 0, 1.2, 0.6], xlim=(0.2, 1))
ax1.errorbar(x=xmm_df.x_xmm, y=xmm_df.y_xmm, xerr=xmm_df.xerr_xmm, yerr=xmm_df.yerr_xmm, fmt='.', elinewidth=0.6, label="XMM - North Aurora (08-04-2020)")
ax1.plot(xmm_df.x_xmm, folded_model, color='orangered', label='Best-fit Model (folded)')
ax1.grid(color='w'), ax1.set_xticklabels([]), ax1.set_ylabel('Counts / s / keV')

handles,labels = ax1.get_legend_handles_labels()
handles = [handles[1], handles[0]]
labels = [labels[1], labels[0]]
ax1.legend(handles,labels)

ax2 = fig.add_axes([0, -0.6, 1.2, 0.6], xlim=(0.2, 1))
ax2.plot(lem_df.x_lem, lem_df.y_lem, label='LEM') 
ax2.grid(color='w'), ax2.set_xlabel('Energy / keV'), ax2.set_ylabel('Counts / s / keV'), ax2.legend()

ax1.set_title("LEM Observation of Jupiter's Equatorial Disk For APEC Model at Solar Minimum")

# Secondary Axis: 
def tick_function(X): 
    V = 12.398/X
    return ["%.3f" % z for z in V]

ax1t = ax1.twiny()
ax1tTicks = ax1.get_xticks()   
ax1t.set_xticks(ax1tTicks)
ax1t.set_xbound(ax1.get_xbound())
ax1t.set_xticklabels(tick_function((ax1tTicks)))
ax1t.set_xlabel('Wavelength ('+r'$\AA$)')

# plt.savefig('lem-solar-min-equator-apec.pdf', bbox_inches='tight')

In [ ]:
# Saving CSVs to manually look for peaks and find out the energy, then to look up the ion on atomdb
# http://www.atomdb.org/Webguide/webguide.php

# xmm_df.to_csv(r'xmm_min_df.csv', index=False)
# lem_df.to_csv(r'lem_min_df.csv', index=False)